#  1. Load and Prepare the Data

In [1]:
import pandas as pd

# Load your data
df = pd.read_csv('../data/yfinance_data/AAPL_historical_data.csv', parse_dates=['Date'])
df.set_index('Date', inplace=True)
df = df.sort_index()

# Ensure required columns are present
required_cols = ['Open', 'High', 'Low', 'Close', 'Volume']
assert all(col in df.columns for col in required_cols), "Missing required columns"
df.head()


,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.098943,469033600,0.0,0.0
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093781,175884800,0.0,0.0
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086898,105728000,0.0,0.0
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089049,86441600,0.0,0.0
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091630,73449600,0.0,0.0


# 2. Calculate Indicators with TA-Lib

In [3]:
import talib

# Simple Moving Average
df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)

# Relative Strength Index
df['RSI'] = talib.RSI(df['Close'], timeperiod=14)

# MACD
df['MACD'], df['MACD_signal'], _ = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9)


ModuleNotFoundError: No module named 'talib'

 # 3. Use PyNance for Financial Metrics

In [2]:
import pynance as pn

# Load from pynance if you want fresh data (optional)
# tsla = pn.data.get('TSLA')
# tsla.df.tail()

# Calculate basic indicators via pynance (optional)
# tsla.sma(period=20)
# tsla.rsi(period=14)


#  4. Visualize the Data

In [1]:
import matplotlib.pyplot as plt

# Closing Price + SMA
plt.figure(figsize=(14, 6))
plt.plot(df['Close'], label='Close')
plt.plot(df['SMA_20'], label='SMA 20')
plt.title('Close Price with SMA 20')
plt.legend()
plt.show()

# RSI
plt.figure(figsize=(14, 4))
plt.plot(df['RSI'], label='RSI')
plt.axhline(70, color='red', linestyle='--')
plt.axhline(30, color='green', linestyle='--')
plt.title('RSI (Relative Strength Index)')
plt.legend()
plt.show()

# MACD
plt.figure(figsize=(14, 6))
plt.plot(df['MACD'], label='MACD')
plt.plot(df['MACD_signal'], label='Signal Line')
plt.title('MACD Indicator')
plt.legend()
plt.show()


NameError: name 'df' is not defined

<Figure size 1400x600 with 0 Axes>